<a href="https://colab.research.google.com/github/jonnells/cmpe255-project-grp/blob/main/Yelp_On_The_Go.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yelp On the Go!

In [1]:
import os
import json

## Import Data

In [2]:
yelp_dataset_path = 'yelp_dataset/'

In [4]:
user_data = []
with open(yelp_dataset_path + 'yelp_academic_dataset_user.json', encoding="utf8") as fin:
    print('Reading yelp_academic_dataset_user.json')
    for line in fin:
        user_data.append(json.loads(line))

Reading yelp_academic_dataset_user.json


In [5]:
review_data = []
with open(yelp_dataset_path + 'yelp_academic_dataset_review.json', encoding="utf8") as fin:
    print('Reading yelp_academic_dataset_review.json')
    for line in fin:
        review_data.append(json.loads(line))

Reading yelp_academic_dataset_review.json


In [29]:
business_data = []
with open(yelp_dataset_path + 'yelp_academic_dataset_business.json', encoding="utf8") as fin:
    print('Reading yelp_academic_dataset_business.json')
    for line in fin:
        business_data.append(json.loads(line))

Reading yelp_academic_dataset_business.json


## Data Investigation

In [31]:
print(str(len(user_data)) + ' user records loaded')
print(str(len(review_data)) + ' review records loaded')
print(str(len(business_data)) + ' business records loaded')

573316 user records loaded
8635403 review records loaded
160585 business records loaded


In [8]:
print(user_data[0].keys())

dict_keys(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny', 'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 'compliment_writer', 'compliment_photos'])


In [9]:
print(review_data[0].keys())

dict_keys(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date'])


In [30]:
print(business_data[0].keys())

dict_keys(['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours'])


## Data Pre-processing

Remove users that do not have enough reviews (10)

In [25]:
user_data = [user for user in user_data if user.get('review_count') >= 15]

In [26]:
print(str(len(user_data)) + ' useful user records')

573316 useful user records


Only keep restaurant related businesses

In [37]:
print(business_data[0])

{'business_id': '6iYb2HFDywm3zjuRg0shjw', 'name': 'Oskar Blues Taproom', 'address': '921 Pearl St', 'city': 'Boulder', 'state': 'CO', 'postal_code': '80302', 'latitude': 40.0175444, 'longitude': -105.2833481, 'stars': 4.0, 'review_count': 86, 'is_open': 1, 'attributes': {'RestaurantsTableService': 'True', 'WiFi': "u'free'", 'BikeParking': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'BusinessAcceptsCreditCards': 'True', 'RestaurantsReservations': 'False', 'WheelchairAccessible': 'True', 'Caters': 'True', 'OutdoorSeating': 'True', 'RestaurantsGoodForGroups': 'True', 'HappyHour': 'True', 'BusinessAcceptsBitcoin': 'False', 'RestaurantsPriceRange2': '2', 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': True}", 'HasTV': 'True', 'Alcohol': "'beer_and_wine'", 'GoodForMeal': "{'dessert': False, 'latenight': Fa

In [58]:
restaurant_category = 'Restaurants'
restaurant_data = []
for business in business_data:
    try:
        if restaurant_category in business.get('categories',):
            restaurant_data.append(business)
    except:
        pass
        

#restaurant_data = [business for business in business_data if restaurant_category in business.get('categories', default=None)]

In [59]:
print(str(len(restaurant_data)) + ' restaurant records')

50763 restaurant records


Grab imporant attributes

In [65]:
restaurant_ids = [restaurant.get('business_id') for restaurant in restaurant_data]

In [53]:
restaurant_stars = [restaurant.get('stars') for restaurant in restaurant_data]

In [54]:
user_ids = [user.get('user_id') for user in user_data]

In [69]:
states = [restaurant.get('state') for restaurant in restaurant_data]
tempset = set(states)
states = list(tempset)
print(states)

['MN', 'GA', 'NH', 'TX', 'OH', 'BC', 'WA', 'WY', 'OR', 'MA', 'FL', 'VA', 'KY', 'KS', 'ABE', 'CO']


In [89]:
MN_restaurants = [restaurant for restaurant in restaurant_data if 'MN' in restaurant.get('state')]
GA_restaurants = [restaurant for restaurant in restaurant_data if 'GA' in restaurant.get('state')]
NH_restaurants = [restaurant for restaurant in restaurant_data if 'NH' in restaurant.get('state')]
TX_restaurants = [restaurant for restaurant in restaurant_data if 'TX' in restaurant.get('state')]
OH_restaurants = [restaurant for restaurant in restaurant_data if 'OH' in restaurant.get('state')]
BC_restaurants = [restaurant for restaurant in restaurant_data if 'BC' in restaurant.get('state')]
WA_restaurants = [restaurant for restaurant in restaurant_data if 'WA' in restaurant.get('state')]
WY_restaurants = [restaurant for restaurant in restaurant_data if 'WY' in restaurant.get('state')]
OR_restaurants = [restaurant for restaurant in restaurant_data if 'OR' in restaurant.get('state')]
MA_restaurants = [restaurant for restaurant in restaurant_data if 'MA' in restaurant.get('state')]
FL_restaurants = [restaurant for restaurant in restaurant_data if 'FL' in restaurant.get('state')]
VA_restaurants = [restaurant for restaurant in restaurant_data if 'VA' in restaurant.get('state')]
KY_restaurants = [restaurant for restaurant in restaurant_data if 'KY' in restaurant.get('state')]
KS_restaurants = [restaurant for restaurant in restaurant_data if 'KS' in restaurant.get('state')]
ABE_restaurants = [restaurant for restaurant in restaurant_data if 'ABE' in restaurant.get('state')]
CO_restaurants = [restaurant for restaurant in restaurant_data if 'CO' in restaurant.get('state')]